University of Michigan - ROB 101 Computational Linear Algebra

# Lab 5: Linear independence, Linear Combinations, and Existence and Uniqueness of Solutions to Ax=b

#### Purpose:  Learn to use Julia to better undersand linear independence and linear combinations
- Skills: 
    - More practice writing functions
    - LU vs LDLT
- Knowledge:
    - Working with Matrix Factorizations
    - Building confidence with complex functions

## Example 1: Finding the Number of Linearly Independent Vectors with LU and LDLT Methods

<img src = "https://i.postimg.cc/g2tJSmPY/ldlt.png" width = 700>

# # of linear independent columns of A is = to # of non-zeros on D (not true in general)

### LDLT Factorization vs LU Factorization

Why do we need to use "enhanced" LU factorization above? Let's explore a (not so uncommon) example of where LU breaks down for the purposes of counting the number of linearly independent vectors.

In [3]:
# Run me, don't change me
using LinearAlgebra
n=4
A=zeros(n,n)
for i=1:n
    if i<n
        A[i,i+1]=1.0
    end
end
@show A

A = [0.0 1.0 0.0 0.0; 0.0 0.0 1.0 0.0; 0.0 0.0 0.0 1.0; 0.0 0.0 0.0 0.0]


4×4 Matrix{Float64}:
 0.0  1.0  0.0  0.0
 0.0  0.0  1.0  0.0
 0.0  0.0  0.0  1.0
 0.0  0.0  0.0  0.0

We see above that A has 3 independent columns, right? Well, let's try and factorize A using LU, and run our "foolproof" method to find how many linearly independent columns there are.

In [4]:
# Run me, don't change me
# A is already upper triangular, but we run LU nevertheless
F=lu(A,check=false)
@show diag(F.U)
U=F.U

diag(F.U) = [0.0, 0.0, 0.0, 0.0]


4×4 Matrix{Float64}:
 0.0  1.0  0.0  0.0
 0.0  0.0  1.0  0.0
 0.0  0.0  0.0  1.0
 0.0  0.0  0.0  0.0

Oh no! The diagonal of U is all zeros, (*the super diagonal is all ones*), so we would be misled into thinking that A has no linearly independent columns, which we know is wrong. How can we fix this?


### The LDLT method 
Let's try to consider $A^\top \cdot A$ instead of just A.

In [5]:
# Run me, don't change me
# A is already upper triangular, but we run LU nevertheless
F=lu(A'*A,check=false)
@show diag(F.U)
U=F.U

diag(F.U) = [0.0, 1.0, 1.0, 1.0]


4×4 Matrix{Float64}:
 0.0  0.0  0.0  0.0
 0.0  1.0  0.0  0.0
 0.0  0.0  1.0  0.0
 0.0  0.0  0.0  1.0

We now see that the diagonal of U is what we would expect it to be - three nonzero elements, indicating three independent columns. But wait, that was still run only using LU factorization! So what's the point of LDLT factorization?

### Let's hunt for a counter example

Below is our implementation of the LDLT algorithm:

In [6]:
# Run me, don't change me!
function ldltROB101(A)
    epsilon = 1e-12
    M = A'*A
    n,m = size(A)
    Areduced = M
    L = Array{Float64,2}(undef,m,0)
    Id = zeros(m,m) + I
    P = Id    
    D=zeros(m,m)
    for i=1:m
        ii=argmax(diag(Areduced[i:m,i:m]))
        mrow=ii[1]+(i-1)
        if !(i==mrow)
            P[[i,mrow],:]=P[[mrow,i],:]
            Areduced[[i,mrow],:]=Areduced[[mrow,i],:]
            Areduced[:,[i,mrow]]=Areduced[:,[mrow,i]]
        end
        if (i>1)
            L[[i,mrow],:] = L[[mrow,i],:]
        end
        pivot=Areduced[i,i]
        if !isapprox(pivot,0,atol=epsilon)
            D[i,i]=pivot
            C=Areduced[:,i]/pivot
            L=[L C]
            Areduced=Areduced-(C*pivot*C')
        else
            L=[L Id[:,i:m]]
            break
        end
    end
    diagD=diag(D)
    return L,P,D,diagD
end

ldltROB101 (generic function with 1 method)

We'll run a loop until we find an example where LU--> diag(U) and LDLT --> diag(D) give different "predictions" for the number of linearly independent columns of A.

*For the record, LDLT is always correct. We are doing this to show that if you try (incorrectly) to use LU for determining the number of linearly independent columns of a matrix, it will sometimes mess up. Hence, we really did need to learn LDLT*

In [7]:
# Run me don't change me
using Random
function CounterExample()
    Random.seed!(4356)
    flag = 1
    N=5
    n=floor(Int,N/2)
    k=0
    while flag > 0
        k=k+1
        # Build a matrix A
        B=randn(N,N-1)
        C=randn(n,n)
        A=[B[:,1:n] B[:,1:n]*C B[:,n+n:end]]
        # Apply LU to A'A
        F=lu(A'*A, check=false)
        diagU=diag(F.U)
        indicesLU=findall(x->x<1e-10, abs.(diagU))
        NumLinIndepLU=N-length(indicesLU)
        # Apply LDLT to A'A
        L,P,D,diagD = ldltROB101(A)
        indicesLDLT=findall(x->x<1e-10, diagD)
        NumLinIndepLDLT=N-length(indicesLDLT)
        # Check for a discrepancy in their reported number of linearly indep columns
        if (NumLinIndepLDLT > NumLinIndepLU)||(k>1e5)
            return k, A, F, L, P, D, diagD
            flag=0
        end        
    end
end

function cleanUp(A,tol=1e-10) #works on matrices or vectors
    # Zero out small entries of a matrix or vector
    B=copy(A)
    indicesSmall=findall(x->x<tol, abs.(B))
    B[indicesSmall]=0.0*B[indicesSmall]
return B
end

cleanUp (generic function with 2 methods)

In [8]:
k, A, F, L, P, D, diagD = CounterExample()

(2, [-1.4535554381481652 -1.0891609802197075 … -2.9814312875497957 1.0645554186689046; 0.9347383417790806 0.5916984480401738 … 1.7092386991712099 -0.6611730059486222; … ; 0.07233432867604413 -0.24806366683725414 … -0.430063735466462 0.7094070636855613; 0.41741684238691484 -1.8478257995519578 … -3.2784741524030956 -0.1413815919499753], LU{Float64, Matrix{Float64}}([7.720117926658887 11.83280360428096 … 27.40888147889123 -4.19498288755786; 0.6923592949053579 -5.882279707321192 … -11.256675928210768 0.6975535510184323; … ; -0.2514608558918805 0.06036761746066256 … -1.7763568394002505e-15 -0.7470590621520714; 0.29925344079454885 -0.3225287408598121 … -0.0 -2.220446049250313e-16], [4, 4, 5, 5, 5], 0), [1.0 -2.801285879557301e-16 … 1.0 0.0; 0.2816648294314558 1.0 … 0.0 0.0; … ; 0.43171420962194007 -0.3225287408598123 … 0.0 0.0; -0.27625350227682033 0.060367617460662736 … 0.0 0.0], [0.0 0.0 … 1.0 0.0; 1.0 0.0 … 0.0 0.0; … ; 0.0 1.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0], [27.40888147889123 0.0 … 0.0 0

In [9]:
# The diagonoal of F.U, showing which elements are nonzero
# run me don't change me
cleanUp(diag(F.U))


5-element Vector{Float64}:
  7.720117926658887
 -5.882279707321192
 -0.0
 -0.0
 -0.0

# Diagonal of LU lies! You cannot get # of independent columns by just applying LU to A'(transpose) A. 

# You must apply LDLT and observe the diagonal of D! 

In [10]:
# The diagonoal of diagD, showing which elements are nonzero
# run me don't change me
cleanUp(diagD)

5-element Vector{Float64}:
 27.40888147889123
  3.1706097052846594
  1.120588593228107
  0.0
  0.0

# 3 non zero entries 

In [11]:
# run me don't change me

# Gives the LDLT factorized upper triangular matrix

cleanUp(D*L')

5×5 Matrix{Float64}:
 27.4089  7.72012  -4.19498  11.8328   -7.5718
 -0.0     3.17061  -1.0253   -1.02261   0.191402
 -0.0     0.0       1.12059  -0.0      -0.0
  0.0     0.0       0.0       0.0       0.0
  0.0     0.0       0.0       0.0       0.0

In [12]:
# Run me don't change me

# Multiplying by transpose(P) on the right of A puts
#   the independent columns to the front of the matrix

# This is very convenient
barA=A*P' 
# P = Permutation matrix from LDLT (ldlt) 
# L = Lower unit triangular matrix
# D = Diagonal matrix
# T or t = transpose

5×5 Matrix{Float64}:
 -2.98143   -1.45356     1.06456    -1.08916    0.786578
  1.70924    0.934738   -0.661173    0.591698  -0.444818
 -2.15989   -1.47617     0.0228749  -0.652562   0.544289
 -0.430064   0.0723343   0.709407   -0.248064   0.130486
 -3.27847    0.417417   -0.141382   -1.84783    0.986634

We see that the first two columns of A are linearly independent, but the first three columns are not:

In [13]:
# Run me don't change me

newA01=A[:,1:3] # Original matrix A
L,P,D,diagD=ldltROB101(newA01)
diagD

3-element Vector{Float64}:
 5.438211460214744
 4.363641102020541
 0.0

We see now that the first three columns of A\*P' are linearly independent

In [14]:
# Run me don't change me

newA02=barA[:,1:3] # All rows, 1-3 (first three) columns of A
L,P,D,diagD=ldltROB101(newA02)
diagD

3-element Vector{Float64}:
 27.40888147889123
  3.1706097052846594
  1.120588593228107

### Bottom line, we can use the LDLT factorization applied to $A^\top \cdot A$ to determine the number of linearly independent columns in A and we form $\bar{A} = A \cdot P^\top$ to move the independent columns to the front of the matrix.

## Problem 1. Number of Linearly Independent Vectors

Using the provided ```ldltROB101``` function, and the above method of determing the number of linearly independent vectors in a matrix, complete the function ```num_independent_vectors``` which takes in a matrix and returns the number of linearly independent vectors.

In [15]:
function num_independent_vectors(A)
    # Your code here!
    L,P,D,diagD = ldltROB101(A) # gives us our matrices
    sum = 0 # set sum to zero
    for i = 1:size(D,1) # D,1. Although diagD is a vector (1 dimension), Julia needs you to clarify because it recognizes it as a tuple
        #  for i = 1:size	for i(any#) =(in range) 1:size(1 to size)
        #  size(A,n)	the size of A along dimension n
        if diagD[i] != 0  # check if we have a value not equal to zero.
            sum += 1  # set sum = to itself + 1 (incriments of 1)
        end
    end
    
    return sum
    
end

num_independent_vectors (generic function with 1 method)

# Building up to you being able to determine if any matrix has a set of "unique" answers.
# Take any "Ax = B" rectangular, with any number of rows and columns (rows != columns), and check in a microsecond:
* whether it has a solution AND
* whether that solution is unique.

In [16]:
# public autograder cell, yay!

A = [1 2 3; 4 5 6; 7 8 10]
a = num_independent_vectors(A)
@show a
T1 = @assert a == 3

B = [1 2 3; 4 5 6; 2 4 6]
b = num_independent_vectors(B)
@show b
T2 = @assert b == 2

@show [T1 T2]

a = 3
b = 2
[T1 T2] = [nothing nothing]


1×2 Matrix{Nothing}:
 nothing  nothing

# We know we wrote our func correctly it right because 2, 4, 6 is a multiple of 1, 2, 3 

## Problem 2. Testing for linear combinations

<img src = "https://i.postimg.cc/2j9KStm1/lin-com-testing.png" width = 700>

You will now use the function you just wrote in order to determine whether a vector is a linear combination of a matrix. Complete the function ```is_linear_combo``` with takes in a vector v and a matrix A and returns true if v is a linear combination of A and false otherwise

In [17]:
function is_linear_combo(A, v)
    
    # return true if v is a linear combination of A, if not then return false
    
end

is_linear_combo (generic function with 1 method)

In [18]:
# public autograder cell, yay!

A = [3.5 1.0 5.0; 5.0 2.0 6.0; 6.5 3.0 7.0; 8.0 4.0 8.0]
v = [4; 4; 4; 4]
T1 = @assert is_linear_combo(A, v)

A = [3.5 1.0 5.0; 5.0 2.0 6.0; 6.5 3.0 7.0; 8.0 4.0 8.0]
v = [20; 11; 12; 14]
T2 = @assert !is_linear_combo(A, v)

@show [T1 T2]

LoadError: TypeError: non-boolean (Nothing) used in boolean context

## Problem 3. Putting it all Together

<img src = "https://i.postimg.cc/MKPM4y72/existence-and-uniqueness.png" width = 700>

You have written a function to find out how many linearly independent vectors are in a matrix. You have written a second function to determine is a vector is a linear combination of a matrix, or set of vectors. Now, finally, you will utilize these two functions to complete the ```exists_and_unique``` function below, which takes in a matrix A and vector b, and returns whether the system Ax = b has a unique solution.

In [19]:
function exists_and_unique(A,b)
    
end

exists_and_unique (generic function with 1 method)

In [20]:
# public autograder cell, yay!

A = [1 2 3; 4 5 6; 7 8 10]
v = [12; 15; 19]
T1 = @assert exists_and_unique(A,v)

@show [T1]

# Test some other matrices!



LoadError: TypeError: non-boolean (Nothing) used in boolean context